# 필요 함수들

In [3]:
# -- 전처리 함수 -- 
import re
import pandas as pd
import string
from emoji import core
import json
from collections import defaultdict
from konlpy.tag import Mecab

def remove_emojis(text):
    return core.replace_emoji(text, replace='')

def remove_punct(text: str) -> str:
    remove_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

def remove_english_and_numbers(text: str) -> str:
    text = re.sub('[a-zㄱ-ㅎ0-9]', '', text).strip() 
    text = re.sub('\s{2,}', ' ', text)
    return text

def remove_numbers(text: str) -> str:
    text = re.sub('[ㄱ-ㅎ0-9]', '', text).strip() 
    text = re.sub('\s{2,}', ' ', text)
    return text

def preprocess_product_name(product_name: str) -> str:
    product_name = remove_punct(product_name)
    # -- 영어 날리는 버전 --
#     product_name = remove_english_and_numbers(product_name)
    # -- 영어 살리는 버전 --
    product_name = remove_numbers(product_name)
    
    return product_name

# -- 전체 데이터 프레임 가져오는 함수 --
def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df

def pull_and_preprocess_base_df():
    base_df = concat_bungae_files()
    base_df['product_name'] = base_df['product_name'].apply(lambda x : preprocess_product_name(x))
    base_df['product_name'] = base_df['product_name'].apply(lambda x : remove_emojis(x))
    return base_df

def bring_fashion_cat_csv():
    """
    번개장터 카테고리 id와 카테고리 이름이 들어있는
    파일 가져오는 함수
    """
    final_cat = pd.read_csv('./final_category.csv')
    final_cat = final_cat.drop('Unnamed: 0', axis=1).copy()
    final_cat['cat_id'] = final_cat['cat_id'].astype(str)
    
    return final_cat

def get_cat_name(cat_id:str) -> str:
    """
    카테고리 id를 입력하면 카테고리 이름을 반환하는 함수 => 중분류에 대해서
    """
    final_cat = bring_fashion_cat_csv()
    if len(cat_id) == 6: # 중분류
        return remove_punct(list(final_cat[final_cat['cat_id'] == cat_id]['cat2'])[0])
    elif len(cat_id) == 9: # 대분류
        return remove_punct(list(final_cat[final_cat['cat_id'] == cat_id]['cat3'])[0])

# -- subs 가 있는 것들만 dictionary로 뽑아내기 --
def make_category_dict_with_subs() -> dict:
    
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)
        
    category_dict_with_subs = dict()
    for main, mids in data.items():
        for mid, subs in mids.items():
            d = defaultdict()
            if subs != [None]: # 하위 sub들이 있을 때 
                d[mid] = subs
            category_dict_with_subs.update(d)
    return category_dict_with_subs


# -- 중분류 내에서의 소분류 morphs 키워드 가져오는 함수 --
def make_subcat_morphs_by_midcat(cat_dict:dict) -> dict:
    """
    make_category_dict_with_subs 함수로 부터 받은 딕션어리로
    midcat의 하위 subcat들의 morphs 키워드 가져오는 함수 
    """
    mecab = Mecab()
    d = defaultdict(list)

    for mid, subs in cat_dict.items():
        tmp_dict = dict()
        for sub in subs:

            mid_morph = mecab.morphs(get_cat_name(mid))
            sub_morph = mecab.morphs(remove_punct(get_cat_name(sub)))
            unique_sub_morph = set(sub_morph) - set(mid_morph)
            unique_sub_morph = list(unique_sub_morph)
            if unique_sub_morph == []:
                d[mid] += sub_morph
            d[mid] += unique_sub_morph

    return dict(d)

# -- 위의 두 함수 합치기 --
def get_dict_from_midcat_with_subs():
    cat_dict = make_category_dict_with_subs()
    morphs_dict = make_subcat_morphs_by_midcat(cat_dict)
    return morphs_dict

def make_category_lists_without_subs():
    """
    소분류 카테고리가 없는 중분류 리스트 => cat_ids_without_subs
    """
    mecab = Mecab()
    # -- 패션 cat_id 들어있는 json 파일 가져오기 --
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)

    cat_list_without_subs = list()
    morphs_list = list()
    for main, mids in data.items():
        for mid in mids:
            if mids[mid] == [None]:
                morphs_list += mecab.morphs(remove_punct(get_cat_name(mid)))
                cat_list_without_subs.append(mid)
    
    morphs_list = list(set(morphs_list))
    
    return morphs_list, cat_list_without_subs

# 번개장터 데이터 

In [ ]:
import pandas as pd
bungae_df = pd.DataFrame()
for idx in range(9):
    df = pd.read_csv(f'./drive-download-20230602T001833Z-001/bungae_df_{idx}_fashion.csv')
    bungae_df = pd.concat([bungae_df, df])
print(bungae_df.shape)
bungae_df.head()

In [ ]:
bungae_df = bungae_df.drop_duplicates('product_id')

In [ ]:
bungae_df = bungae_df.reset_index(drop=True)

In [ ]:
bungae_df.loc[0, 'image_url']

In [ ]:
import urllib.request
prd_id = bungae_df.loc[0, 'product_id']
cat_id = bungae_df.loc[0, 'cat_id']
url = bungae_df.loc[0, 'image_url']
urllib.request.urlretrieve(url, f'./{prd_id}_image.jpg')

In [ ]:
bungae_df.head()

# 카테고리 매칭

In [ ]:
cat_df = pd.read_csv('./final_category.csv')
cat_df = cat_df.drop('Unnamed: 0', axis=1)
cat_df['cat_id'] = cat_df['cat_id'].astype(str)

In [ ]:
cat_df

In [ ]:
cat_with_subs_dict = get_dict_from_midcat_with_subs()
_, cat_without_subs_list = make_category_lists_without_subs()
cat_with_subs_list = list(cat_with_subs_dict.keys())

In [ ]:
cat_without_subs_list

In [ ]:
cat_without_subs_list

In [ ]:
cat_subs_list = list()
for idx in range(len(cat_df)):
    if len(cat_df.loc[idx, 'cat_id']) == 9:
        cat_subs_list.append(cat_df.loc[idx, 'cat_id'])

In [ ]:
bungae_all_cat_id_list = cat_subs_list + cat_without_subs_list

In [ ]:
len(bungae_all_cat_id_list)

# 폴더 만들기

In [ ]:
# -- 위의 카테고리 리스트에 따라 폴더 만들기 --
# -- makedirs가 경로안에 폴더 만들어주는 함수 --
import os
for cat_id in bungae_all_cat_id_list:
    os.makedirs(f'bungae_fashion_image/{cat_id}')

In [ ]:
with open('BGZT_fashion_product_id_list.txt', 'r') as f: # 패션 분야로 변경
        data = f.read()
    prd_id_list = data.split(',')

In [ ]:
# -- 함수로 표현하기 --
with open('./bungae_cat_id.txt', 'r') as file: 
    data = file.read()
cat_id_list = data.split(',')


In [ ]:
import os

def make_folders_for_image():
    with open('./bungae_cat_id.txt', 'r') as file: 
        data = file.read()
    cat_id_list = data.split(',')
    
    for cat_id in cat_id_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'./bungae_fashion_image/{cat_id}')
        
    return 

if __name__ == '__main__':
    make_folders_for_image

# 크롤링 코드

In [ ]:
# -- 간단한 전처리 --
bungae_df = bungae_df.dropna(axis=0)
bungae_df['cat_id'] = bungae_df['cat_id'].astype(int).astype(str)
bungae_image_df = bungae_df[['product_id', 'product_name', 'image_url', 
                             'image_cnt', 'cat_id']]
# -- csv 파일 저장 --
bungae_image_df.to_csv('./bungae_image_df.csv', index=False)

In [ ]:
# -- 베이스 코드 --
import urllib.request
prd_id = bungae_image_df.loc[0, 'product_id']
cat_id = bungae_image_df.loc[0, 'cat_id']
url = bungae_image_df.loc[0, 'image_url']
urllib.request.urlretrieve(url, f'./bungae_fashion_image/{cat_id}/{prd_id}_image.jpg')

In [ ]:
# multiprocessing을 위해 product_id 리스트 텍스트 파일로 만들어주기
bungae_prd_id_list = [str(prd_id) for prd_id in list(bungae_image_df['product_id'])]
with open('./bungae_prd_id.txt', 'w+') as file: # fashion 분야로 축소
        file.write(','.join(bungae_prd_id_list))

In [ ]:
# -- image_url의 형태가 예상한것과 달라서 list를 이용한 multiprocessing이 어려울 것 같음 -- 
import urllib.request
import time
import pandas as pd

def crawl_images_and_put_in_folders(bungae_image_df):
    
    removed_prd_id_list = list()
    
    start = time.time()
    
    for idx in range(len(bungae_image_df)):
        url = bungae_image_df.loc[idx, 'image_url']
        cat_id = bungae_image_df.loc[idx, 'cat_id']
        prd_id = bungae_image_df.loc[idx, 'product_id']
        # -- 사라진 상품들이 있을 수 있기 때문에 try/except로 확인 필요 -- 
        try:
            urllib.request.urlretrieve(url, f'./bungae_fashion_image/{cat_id}/{prd_id}_image.jpg')
            end = time.time()
            print(f'{prd_id} image crawled')
        except:
            removed_prd_id_list.append(prd_id)
            pass 
    
    end = time.time()
    print(f'----- time lapsed : {end-start} -----')
    
    print(f'------ 사라진 상품 갯수 : {len(removed_prd_id_list)} --------', )
    
    # -- 사라진 상품 product_id 리스트 --
    # -- 나중에 해당 상품들은 없애야할 것으로 보임
    with open('./bungae_removed_prd_id.txt', 'w+') as file: 
        file.write(','.join(removed_prd_id_list))
        
    return 

In [ ]:
for idx in range(5):
    url = bungae_image_df.loc[idx, 'image_url']
    cat_id = bungae_image_df.loc[idx, 'cat_id']
    prd_id = bungae_image_df.loc[idx, 'product_id']
    try:
        urllib.request.urlretrieve(url, f'./bungae_fashion_image/{cat_id}/{prd_id}_image.jpg')
        end = time.time()
        print(f'{prd_id} image crawled')
    except:
        removed_prd_id_list.append(prd_id)
         

# 크롤링 코드 2

In [2]:
# -- 생각해보니 대중소로 나누어서 할 경우가 필요할 것이라 생각이 듬 --
import pandas as pd
bungae_image_df = pd.read_csv('./bungae_image_df.csv')
bungae_image_df.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,product_id,product_name,image_url,image_cnt,cat_id
0,201926367,여성야상패딩,https://media.bunjang.co.kr/product/201926367_...,5.0,310090050
1,173779753,여성파라점퍼스XL,https://media.bunjang.co.kr/product/173779753_...,8.0,310090050
2,213485472,경량 패딩 새상품 가니 점퍼 스타일,https://media.bunjang.co.kr/product/213485472_...,10.0,310090050
3,219480633,"휠라 롱패딩,아디다스 패딩, 양털 점퍼, 코트 일괄",https://media.bunjang.co.kr/product/219480633_...,12.0,310090050
4,202566238,무료배송)날씬해보이는 거위솜털 여성패딩점퍼55,https://media.bunjang.co.kr/product/202566238_...,7.0,310090050


In [5]:
cat_with_subs_dict = get_dict_from_midcat_with_subs()
_, cat_without_subs_list = make_category_lists_without_subs()
cat_with_subs_list = list(cat_with_subs_dict.keys())

In [9]:
bungae_image_df['cat_id'] = bungae_image_df['cat_id'].astype(str)
bungae_image_df['maincat_id'] = bungae_image_df['cat_id'].map(lambda x : x[:3])
bungae_image_df['midcat_id'] = bungae_image_df['cat_id'].map(lambda x : x[:6])

In [15]:
for idx in range(len(bungae_image_df)):
    if len(bungae_image_df.loc[idx, 'cat_id']) == 9:
        bungae_image_df.loc[idx, 'subcat_id'] = bungae_image_df.loc[idx, 'cat_id']


In [16]:
bungae_image_df

,product_id,product_name,image_url,image_cnt,cat_id,maincat_id,midcat_id,subcat_id
0,201926367,여성야상패딩,https://media.bunjang.co.kr/product/201926367_...,5.0,310090050,310,310090,310090050
1,173779753,여성파라점퍼스XL,https://media.bunjang.co.kr/product/173779753_...,8.0,310090050,310,310090,310090050
2,213485472,경량 패딩 새상품 가니 점퍼 스타일,https://media.bunjang.co.kr/product/213485472_...,10.0,310090050,310,310090,310090050
3,219480633,"휠라 롱패딩,아디다스 패딩, 양털 점퍼, 코트 일괄",https://media.bunjang.co.kr/product/219480633_...,12.0,310090050,310,310090,310090050
4,202566238,무료배송)날씬해보이는 거위솜털 여성패딩점퍼55,https://media.bunjang.co.kr/product/202566238_...,7.0,310090050,310,310090,310090050
...,...,...,...,...,...,...,...,...
1391017,164097143,전통매듭목걸이/목걸이/전통목걸이/쥬얼리/매듭,https://media.bunjang.co.kr/product/164097143_...,1.0,400999,400,400999,NaN
1391018,174926618,인조모 가발 붙임머리 긴머리,https://media.bunjang.co.kr/product/174926618_...,2.0,400999,400,400999,NaN
1391019,219603119,마리떼 헤어클립 구해요,https://media.bunjang.co.kr/product/219603119_...,1.0,400999,400,400999,NaN
1391020,224250961,새제품)호피 헤어밴드,https://media.bunjang.co.kr/product/224250961_...,1.0,400999,400,400999,NaN


***폴더 다시 만들기***

In [ ]:
# -- 폴더 다시 만들기 --
import os
for cat_id in bungae_all_cat_id_list:
    os.makedirs(f'bungae_fashion/{cat_id}')

In [17]:
bungae_image_df['maincat_id'].unique()

array(['310', '320', '405', '430', '420', '400', '700', '810', '999',
       '100'], dtype=object)

In [29]:
# -- midcat에 3자리 cat_id가 들어있는 것이 있음 --
# -- 이 데이터들은 삭제 --
idx_list = list()
for idx in range(len(bungae_image_df)):
    if (len(bungae_image_df.loc[idx, 'midcat_id']) < 6) |\
                (bungae_image_df.loc[idx, 'midcat_id'][0] not in ['3', '4']):
        idx_list.append(idx)
idx_list

In [40]:
# -- 패션 카테고리에 들어있지 않은 상품들 제거 --
bungae_image_df = bungae_image_df.drop(bungae_image_df.index[idx_list], axis=0)

In [42]:
# -- reset index --
bungae_image_df = bungae_image_df.reset_index(drop=True)

In [46]:
# -- maincat, midcat 리스트 --
maincat_list = list(bungae_image_df['maincat_id'].unique())
midcat_list = list(bungae_image_df['midcat_id'].unique())

array(['310', '320', '405', '430', '420', '400'], dtype=object)

In [73]:
# -- 폴더 다시 만들기 --
# -- 순서 : 대분류 > 중분류 > 소분류 --
# -- 소분류 없으면 중분류까지만 -- 
import os

# -- 가장 상단 파일 : bungae_images --
# -- 나중에 파일 이름 바꿔야 될 수 있음 -- 
for idx in range(len(bungae_image_df)):
    maincat = bungae_image_df.loc[idx, 'maincat_id']
    midcat = bungae_image_df.loc[idx, 'midcat_id']
    subcat = bungae_image_df.loc[idx, 'subcat_id']
    
    if type(subcat) == str:
        file_path = f'./bungae_images/{maincat}/{midcat}/{subcat}'
        if os.path.exists(f'./bungae_images/{maincat}/{midcat}/{subcat}'):
            pass
        else:
            os.makedirs(f'./bungae_images/{maincat}/{midcat}/{subcat}')
    elif type(subcat) == float :
        file_path = f'./bungae_images/{maincat}/{midcat}/{subcat}'
        if os.path.exists(f'./bungae_images/{maincat}/{midcat}'):
            pass
        else:
            os.makedirs(f'./bungae_images/{maincat}/{midcat}')

In [ ]:
# -- 다시 크롤링? OR 크롤링한 파일에서 move 하기? -- 
# 1. move해보기
# 2. 안될 것 같으면 다시 크롤링 